In [1]:
using DataFrames, Distributions, DataFramesMeta

In [2]:
function gen_data(
    ; 
    n = 500000, 
    mu = [0, 2], 
    sigma = 1, 
    pZ1 = 0.8)
    
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [3]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,0.592501,0.334718,0.14816
2,2,-0.997777,0.242509,0.00446149
3,3,-2.2522,0.0315828,4.72743e-5
4,4,1.46137,0.137142,0.345073
5,5,1.24542,0.183696,0.300101
6,6,-0.312897,0.379883,0.0274968


In [4]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.00001,
    max_iterations = 1000,
    progress = DataFrame(
        Iteration = 0:max_iterations, 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        loglik = Vector{Float64}(undef, max_iterations+1), 
        diff_loglik = Vector{Float64}(undef, max_iterations+1)
        ),
    verbose = false
    )

    # initial E step, to perform needed calculations for initial likelihood:
    E_step!(data, pi_hat_0) 
    ll = loglik(data)
    progress[1, :] = (0, pi_hat_0, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations

        # M step: re-estimate parameters
        pi_hat = M_step(data)

        # E step: re-compute distribution of missing variables, using parameters
        E_step!(data, pi_hat)
        
        # Assess convergence

        ## save the previous log-likelihood so we can test for convergence
        ll_old = ll
        
        ## here's the new log-likelihood
        ll = loglik(data)

        ll_diff = ll - ll_old
        
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)
        
        if(verbose)
            print(progress[i+1,:])
        end
        
        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    
    return progress[1:(last_iter + 1), :]
end

fit_model! (generic function with 1 method)

In [5]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [6]:
function M_step(data)
    data[:, :pZ1_given_Y] |> mean
end

M_step (generic function with 1 method)

In [7]:
function loglik(data)
    data[:, :pY] .|> log |> sum
end

loglik (generic function with 1 method)

In [8]:
@time progress = fit_model!(data, tolerance = 0.00001);


The first run requires compilation; on subsequent calls it will be faster:

In [ ]:
@time progress = fit_model!(data, tolerance = 0.00001)

  0.145387 seconds (13.08 k allocations: 504.204 MiB, 15.44% gc time)


,Iteration,pi_hat,loglik,diff_loglik
,Int64,Float64,Float64,Float64
1,0,0.5,-8.77764e5,NaN
2,1,0.665587,-8.37076e5,40687.3
3,2,0.737939,-8.2809e5,8986.3
4,3,0.770411,-8.26009e5,2080.74
5,4,0.78577,-8.25502e5,506.877
6,5,0.793294,-825375.0,127.46
7,6,0.797054,-8.25342e5,32.6188
8,7,0.798953,-825334.0,8.42551
9,8,0.799917,-8.25332e5,2.18686
